In [63]:
import Levenshtein
import pandas as pd
import re
import os
%pip install pyxDamerauLevenshtein
%pip install cython

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [64]:
import regex as rex
import nltk

In [4]:
answer = pd.read_excel(r"C:/Users/HP/Desktop/itn/nlpTest/模糊比對/testData/2023ICD10.xlsx", sheet_name = 0, usecols = [0, 3], engine='openpyxl')

In [66]:
import csv

csv_file_path = 'C:/Users/HP/Desktop/itn/nlpTest/hugNLP/testData/csv_POC1.csv'

with open(csv_file_path, 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    fieldnames = reader.fieldnames

print(fieldnames[1])


OCR_result


In [112]:
#用jieba分詞
import jieba


# 開啟 CSV 檔案
with open(r'C:/Users/HP/Desktop/itn/nlpTest/hugNLP/testData/csv_POC1.csv', 'r', encoding = 'utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        text = row['OCR_result']
        # 分詞
        tokens = jieba.lcut(text, cut_all = False)

        # 清理數字和標點符號
        #clean_tokens = [token for token in tokens if re.match(r" ^[^\d\W]+$", token)]

        # 列印結果
        print("原始文本:", text)
        print("分詞結果:", tokens)
        

原始文本: 光田醫療
分詞結果: ['光田', '醫療']
原始文本: 光田綜合醫院
分詞結果: ['光田', '綜合', '醫院']
原始文本: 診證明書
分詞結果: ['診證', '明書']
原始文本: 面般
分詞結果: ['面般']
原始文本: Stanaard
分詞結果: ['Stanaard']
原始文本: 社團法
分詞結果: ['社團法']
原始文本: 回傷害
分詞結果: ['回傷害']
原始文本: Identi fication
分詞結果: ['Identi', ' ', 'fication']
原始文本: of
分詞結果: ['of']
原始文本: Injur ies
分詞結果: ['Injur', ' ', 'ies']
原始文本: Cerfificate
分詞結果: ['Cerfificate']
原始文本: of
分詞結果: ['of']
原始文本: Diagnosis
分詞結果: ['Diagnosis']
原始文本: 圓天車Daa
分詞結果: ['圓天車', 'Daa']
原始文本: 回獎役專用
分詞結果: ['回獎役', '專用']
原始文本: Mi l itary
分詞結果: ['Mi', ' ', 'l', ' ', 'itary']
原始文本: 沙鹿Sha1u
分詞結果: ['沙鹿', 'Sha1u']
原始文本: 口大陸地區人民來合探病用
分詞結果: ['口大陸', '地區', '人民', '來合', '探病', '用']
原始文本: Mainland
分詞結果: ['Mainland']
原始文本: 0007
分詞結果: ['0007']
原始文本: 0665
分詞結果: ['0665']
原始文本: 病歷號碼
分詞結果: ['病歷', '號碼']
原始文本: Medical.
分詞結果: ['Medical', '.']
原始文本: RecordNumbet
分詞結果: ['RecordNumbet']
原始文本: 姓名
分詞結果: ['姓名']
原始文本: 性別
分詞結果: ['性別']
原始文本: 出生自期了
分詞結果: ['出生', '自期', '了']
原始文本: 李
分詞結果: ['李']
原始文本: 華
分詞結果: ['華']
原始文本: 民國
分詞結果: ['民國']
原始文本: 52
分詞結果: ['52']


修改pattern
增加計算距離的條件
排出ranking
用CKIP把answer跟text都做分詞之後再進行查找

先剔除全無中文的部分  對查找的answer做修改  對ICD10做分詞、分類?(做unique是啥)  [用PANDAS對字詞作分類]

分兩邊(text, answer)想辦法修得更精

用gpu跑一次

In [3]:
##question-answering
from transformers import pipeline, BertTokenizerFast, AutoModel, Pipeline
import re
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
import json

def test_model(task: str, model_name = str, tokenizer = None):
    if tokenizer is None:
        generator = pipeline(task = task, model = model_name)
    else:
        generator = pipeline('token-classification', model = model_name, tokenizer = tokenizer)
    return generator

def qa_format(generator: Pipeline, questions: list, context: str):
    result = {}
    for question in questions:
        predict = generator(question = question, 
                            context = context, 
                            nax_answer_len = 150, 
                            top_k = 3)
        pred_format = ['('+str(round(pred['score']*100, 2)) + '%)' + pred['answer'] for pred in predict]
        result[question] = pred_format
    return result

def ckip_tool(use_delim = False):
    global input
    ws_driver = CkipWordSegmenter(model = 'bert-base')
    pos_driver = CkipPosTagger(model = 'bert-base')
    ner_driver = CkipNerChunker(model = 'bert-base')
    
    #run pipeline
    ws = ws_driver(input, use_delim = use_delim)
    pos = pos_driver(ws, use_delim = use_delim)
    ner = ner_driver(input, use_delim = use_delim)
    
    #results
    result = ''
    for sentence, sentence_ws, sentence_pos, sentence_ner in zip(input, ws, pos, ner):
        result += sentence + '\n'
        result += pack_ws_pos_sentence(sentence_ws, sentence_pos) + '\n'
        for entity in sentence_ner:
            result += str(entity) + '\n'
        result += '\n'
    return result

#pack word segmentation and part-of-speech results
def pack_ws_pos_sentence(sentence_ws, sentence_pos):
    assert len(sentence_ws) == len(sentence_pos)
    res = []
    for word_ws, word_pos, in zip(sentence_ws, sentence_pos):
        res.append(f'{word_ws}({word_pos})')
        return '\u3000'.join(res)
    
#ens_input = 'print date: 1/19/2018 Taipei City United! Hospital (Heping District) Diagnosis Certificate No.3 Surname Wang MeiqiGender ID

with open('C:/Users/HP/Desktop/itn/nlpTest/模糊比對/testData/csv_POC1.csv', 'r', encoding = 'utf8') as txtFile:
    text = txtFile.read()

##for ckip tool
#text = ''.join(re.findall(r'[\u4e00-\u9fff]+', text))
#input = [text]
## for qa
text = text.replace('\n', '。').replace('\t', ' ')
input = text

with open('C:/Users/HP/Desktop/itn/nlpTest/模糊比對/mdeberta-multilingual.json', 'w', encoding = 'utf8') as jsonFile:
    generator = test_model('question-answering', 'timpal0l/mdeberta-v3-base-squad2')
    json.dump(qa_format(generator, ['病名是什麼?', '醫囑有哪些'], input), 
              txtFile, 
              ensure_ascii=False, 
              indent=4)
    

    

ValueError: I/O operation on closed file.

In [39]:
#用CKIP分詞完之後作為answer，再進行模糊比對

from transformers import pipeline, BertTokenizerFast, AutoModel, Pipeline
import re
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
import json

def ckip_tool(use_delim = True):
    ws_driver = CkipWordSegmenter(model = 'bert-base')
    pos_driver = CkipPosTagger(model = 'bert-base')
    result = []
    
    #run pipeline
    for _, rows in answer.iterrows():
        text_in_answer = rows["2023 CM中文名稱"]
        ws = ws_driver(rows, use_delim = use_delim)
        pos = pos_driver(ws, use_delim = use_delim)

    
    #results
    
    for sentence, sentence_ws, sentence_pos in zip(rows, ws, pos):
        result += sentence + '\n'
        result += pack_ws_pos_sentence(sentence_ws, sentence_pos) + '\n'
        result += '\n'
    return result

#pack word segmentation and part-of-speech results

def pack_ws_pos_sentence(sentence_ws, sentence_pos):
    assert len(sentence_ws) == len(sentence_pos)
    res = []
    for word_ws, word_pos, in zip(sentence_ws, sentence_pos):
        res.append(f'{word_ws}({word_pos})')
        return '\u3000'.join(res)
    

In [41]:
result = ckip_tool()

print(result)

Tokenization: 100%|██████████| 2/2 [00:00<00:00, 1003.90it/s]


In [9]:
##還缺去除全英文情形，levenshtein(計算字串替換距離) or fuzzywuzzy進行字串比對 / pandas撈

import csv
import re
import pandas as pd

# 開啟 csv 檔案
with open(r'C:/Users/HP/Desktop/itn/nlpTest/模糊比對/testData/csv_POC1.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)

    # 建立空的 DataFrame
    result_df = pd.DataFrame(columns=["original_text", "matched_text"])

    for row in reader:
        # 進行逐筆資料比對
        line = row['OCR_result']
        pattern = re.compile(line.strip())  # 使用 csv 檔案中的資料作為正則表達式
        found = False  # 用來標記是否在 answer 中找到匹配的資料

        for _, row in answer.iterrows():
            text_in_answer = row["2023 CM中文名稱"]  # 假設 answer 中的文本資料在 'text' 欄位中

            # 使用 re.findall 進行比對並輸出所有結果
            match = pattern.findall(text_in_answer)
            if match:
                print(text_in_answer)
                found = True

                # 將找到的匹配資料加入到 DataFrame 中
                new_data = pd.DataFrame({'original_text': [line], 'matched_text': [text_in_answer]})
                result_df = pd.concat([result_df, new_data], ignore_index=True)

        if not found:
            continue

# 輸出結果的 DataFrame
print(result_df)


Nezelof氏症候群
Sandhoff氏疾病
嬰兒型脊髓性肌肉萎縮症, 第I型 [Werdnig-Hoffmann氏病]
Nezelof氏症候群
Sandhoff氏疾病
嬰兒型脊髓性肌肉萎縮症, 第I型 [Werdnig-Hoffmann氏病]
性別認同障礙症
兒童期性別認同障礙症
其他性別認同障礙症
非特定的性別認同障礙症
性別不明及假性陰陽人
性別不明
非二元性別伴侶為加害者之不當對待及疏忽
現任非二元性別伴侶為加害者之不當對待及疏忽
前任非二元性別伴侶為加害者之不當對待及疏忽
李斯特菌病
皮膚李斯特菌病
李斯特菌性腦膜炎及腦膜腦炎
李斯特菌性腦膜炎
李司特菌性腦膜腦炎
李司特菌性敗血症
其他型李斯特菌
眼腺體性李司特菌病
李司特菌性心內膜炎
其他型李司特菌病
李司特菌病
新生兒(散播性)李司特菌病
李霍氏骨折
李霍氏 I 型骨折
李霍氏I型閉鎖性骨折之初期照護
李霍氏I型開放性骨折之初期照護
李霍氏I型骨折癒合之後續照護
李霍氏I型骨折延遲癒合之後續照護
李霍氏I型骨折未癒合之後續照護
李霍氏I型骨折之後遺症
李霍氏 II型骨折
李霍氏II型閉鎖性骨折之初期照護
李霍氏II型開放性骨折之初期照護
李霍氏II型骨折癒合之後續照護
李霍氏II型骨折延遲癒合之後續照護
李霍氏II型骨折未癒合之後續照護
李霍氏II型骨折之後遺症
李霍氏III型骨折
李霍氏III型閉鎖性骨折之初期照護
李霍氏III型開放性骨折之初期照護
李霍氏III型骨折癒合之後續照護
李霍氏III型骨折延遲癒合之後續照護
李霍氏III型骨折未癒合之後續照護
李霍氏III型骨折之後遺症
肝吸蟲病(中華肝吸蟲病)
Waldenstrom(華登特倫氏)巨大球蛋白血症
伏華卡巴-邁爾-史密斯症候群
月經性偏頭痛，非頑固性
月經性偏頭痛，非頑固性，伴有偏頭痛重積狀態
月經性偏頭痛，非頑固性，未伴有偏頭痛重積狀態
月經性偏頭痛，頑固性
月經性偏頭痛，頑固性，伴有偏頭痛重積狀態
月經性偏頭痛，頑固性，未伴有偏頭痛重積狀態
半月板囊腫
未明示之半月板囊腫
右側膝部外側未明示之半月板囊腫
左側膝部外側未明示之半月板囊腫
未明示側性膝部外側未明示之半月板囊腫
右側膝部內側未明示之半月板囊腫
左側膝部內側未明示之半月板囊腫
未明示側性膝部內側未明示之半月板囊腫
右側膝部未明示之半月板囊腫
左側膝部未明示之半月板囊腫
未

In [39]:
##複製備用

import json

# 讀取 JSON 檔案
with open("C:/Users/HP/Desktop/itn/nlpTest/模糊比對/mdeberta-multilingual.json", "r") as json_file:
    data = json.load(json_file)

# 將列表中的元素轉換為字典
data_dict = {}
for item in data:
    key = item
    value = data[item]
    data_dict[key] = value
    
for key in data.keys():
    values = data[key]
    

key = "病名是什麼?"
first_value = data[key][0]

answer = pd.read_excel(r"C:/Users/HP/Desktop/itn/nlpTest/模糊比對/testData/2023ICD10.xlsx", sheet_name = 0, usecols = [0, 3], engine='openpyxl')

import csv
import re
import pandas as pd

    # 建立空的 DataFrame
result_df = pd.DataFrame(columns=["original_text", "matched_text"])

pattern = re.compile(first_value.strip())  # 使用 csv 檔案中的資料作為正則表達式
found = False  # 用來標記是否在 answer 中找到匹配的資料

for _, row in answer.iterrows():
    text_in_answer = row["2023 CM中文名稱"]  # 假設 answer 中的文本資料在 'text' 欄位中

    # 使用 re.findall 進行比對並輸出所有結果
    match = pattern.findall(text_in_answer)
    
    if match:
        print(text_in_answer)
        found = True

        # 將找到的匹配資料加入到 DataFrame 中
        new_data = pd.DataFrame({'original_text': [first_answer], 'matched_text': [text_in_answer]})
        result_df = pd.concat([result_df, new_data], ignore_index=True)

    if not found:
        continue

# 輸出結果的 DataFrame
print(result_df)

Empty DataFrame
Columns: [original_text, matched_text]
Index: []


In [41]:
answer = pd.read_excel(r"C:/Users/HP/Desktop/itn/nlpTest/模糊比對/testData/2023ICD10.xlsx", sheet_name = 0, usecols = [0, 3], engine='openpyxl')
test1 = '直腸惡性腫瘤'
import csv
import re
import pandas as pd

# 建立空的 DataFrame
result_df = pd.DataFrame(columns=["original_text", "matched_text"])

pattern = re.compile(test1)  # 使用 csv 檔案中的資料作為正則表達式
found = False  # 用來標記是否在 answer 中找到匹配的資料

for _, row in answer.iterrows():
    text_in_answer = row["2023 CM中文名稱"]  # 假設 answer 中的文本資料在 'text' 欄位中

    # 使用 re.findall 進行比對並輸出所有結果
    match = pattern.findall(text_in_answer)
    
    if match:
        print(text_in_answer)
        found = True

        # 將找到的匹配資料加入到 DataFrame 中
        new_data = pd.DataFrame({'original_text': [test1], 'matched_text': [text_in_answer]})
        result_df = pd.concat([result_df, new_data], ignore_index=True)

    if not found:
        continue

# 輸出結果的 DataFrame
print(result_df)

直腸惡性腫瘤
來院接受直腸惡性腫瘤之篩檢
  original_text   matched_text
0        直腸惡性腫瘤         直腸惡性腫瘤
1        直腸惡性腫瘤  來院接受直腸惡性腫瘤之篩檢
